# IMDB 영화평 감성분석
 - pipeline
 -TfidfVectorizer + LogisticRegression

In [1]:
import pandas as pd
import numpy as np
from google.colab import files
up = files.upload()

Saving labeledTrainData.tsv to labeledTrainData.tsv


In [2]:
df = pd.read_csv('labeledTrainData.tsv', sep='\t', quoting=3)   # 3: QUOTE_NONE
df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


 - 텍스트 전처리

In [3]:
# <br /> 태그는 공백으로 변환
df.review = df.review.str.replace('<br />',' ')

import warnings
warnings.filterwarnings('ignore')

# 구둣점, 숫자 제거 - 영문자가 아닌 글자는 공백으로 변환
# 데이터프레임의 str method는 정규표현식을 지원함
df.review = df.review.str.replace('[^A-Za-z]', ' ').str.strip()
df.review[0][:1000]

'With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for  

In [4]:
# 학습
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.review, df.sentiment, stratify=df.sentiment, random_state=2022)

 - Pipeline : TfidfVectorizer + Logistic Regression

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [6]:
# Pipeline으로 넣어주는 작업 (vectorize)
tv = TfidfVectorizer(ngram_range=(1,2), stop_words='english')
lrc = LogisticRegression(random_state=2022)
pipeline = Pipeline([("TVECT", tv), ('LR', lrc)])

In [7]:
# 학습
%time pipeline.fit(X_train, y_train)

CPU times: user 35.1 s, sys: 15 s, total: 50.1 s
Wall time: 33.2 s


Pipeline(steps=[('TVECT',
                 TfidfVectorizer(ngram_range=(1, 2), stop_words='english')),
                ('LR', LogisticRegression(random_state=2022))])

In [8]:
# 평가
pipeline.score(X_test, y_test)

0.87472

 - 최적 파라미터 찾기

In [9]:
from sklearn.model_selection import GridSearchCV
params = {'TVECT__max_df': [100,600], 'LR__C': [6,12]}

In [10]:
grid_pip = GridSearchCV(pipeline, params, scoring='accuracy', cv=3, n_jobs=-1) # n_jobs 병행처리
%time grid_pip.fit(X_train, y_train)

CPU times: user 32.4 s, sys: 13.2 s, total: 45.6 s
Wall time: 3min 19s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('TVECT',
                                        TfidfVectorizer(ngram_range=(1, 2),
                                                        stop_words='english')),
                                       ('LR',
                                        LogisticRegression(random_state=2022))]),
             n_jobs=-1,
             param_grid={'LR__C': [6, 12], 'TVECT__max_df': [100, 600]},
             scoring='accuracy')

In [11]:
grid_pip.best_params_

{'LR__C': 12, 'TVECT__max_df': 600}

In [12]:
grid_pip.best_estimator_.score(X_test, y_test)

0.87728

 - 모델 저장하고 불러오기

In [14]:
import joblib
joblib.dump(grid_pip.best_estimator_, 'imdb_tvect_lr.pkl')

['imdb_tvect_lr.pkl']

In [15]:
best_pipe = joblib.load('imdb_tvect_lr.pkl')

 - 실제 데이터 적용

In [16]:
rv ='''Everything about this movie is trying too hard - the over dramatic score, the long shots on characters faces, the overacting, the complex crime story - it all feels it's trying to get an Oscar in every moment.
It's overly long, drawn out, and the story feels like a generic crime saga that has the Batman universe shoehorned into it.
This movie is not a masterpiece, but it spends a lot of effort making you think it is!'''

In [17]:
import re
rv = re.sub('[^A-Za-z]',' ', rv).strip()

In [18]:
best_pipe.predict([rv])

array([0])